In [1]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.colors as pc
from modified_spectral_method import recursive_spectral_method, create_correlation_matrix, calculate_C_g
from modified_louvain_method import *
from itertools import product
from collections import Counter

In [5]:
df = pd.read_csv('sens_data/ITRAXX-Europe Timeseries 20241127.csv')
df2 = pd.read_csv('sens_data/eur_data_standardized_returns.csv')

In [19]:
# Convert the 'AsOf' column to a datetime object
df['Date'] = pd.to_datetime(df['Date'], format='%d/%b/%y')

# Sort the DataFrame by the 'AsOf' column in descending order by day
df_sorted = df.sort_values(by='Date', ascending=True)

In [27]:
df_filtered = df_sorted[df_sorted['Date'].isin(df2['Date'])]
df_log_returns = df_filtered.copy()

# Calculate the log returns for the CDS spread
df_log_returns['Log_Return'] = np.log(df_filtered['ConvSpread'] / df_filtered['ConvSpread'].shift(1))

# Drop NaN values that occur due to the first row (no previous value to compare)
df_log_returns = df_log_returns.dropna(subset=['Log_Return'])

# Standardize the log returns (Z-score normalization)
log_return_mean = df_log_returns['Log_Return'].mean()
log_return_std = df_log_returns['Log_Return'].std()
df_log_returns['Standardized_Log_Return'] = (df_log_returns['Log_Return'] - log_return_mean) / log_return_std

# Display the filtered DataFrame
df_log_return

C:\Users\iChin\AppData\Local\Temp\ipykernel_39452\1092003119.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df_filtered = df_sorted[df_sorted['Date'].isin(df2['Date'])]


,Date,Batch,Tenor,Ticker,Series,Version,ConvSpread,Log_Return,Standardized_Log_Return
3700,2010-09-16,EOD,5Y,ITRAXX-Europe,13,1,106.28,0.006608,0.214348
3699,2010-09-17,EOD,5Y,ITRAXX-Europe,13,1,106.96,0.006378,0.206844
3698,2010-09-20,EOD,5Y,ITRAXX-Europe,14,1,110.97,0.036805,1.197993
3697,2010-09-21,EOD,5Y,ITRAXX-Europe,14,1,109.85,-0.010144,-0.331349
3696,2010-09-22,EOD,5Y,ITRAXX-Europe,14,1,114.08,0.037784,1.229890
...,...,...,...,...,...,...,...,...,...
1192,2020-04-28,EOD,5Y,ITRAXX-Europe,33,1,81.58,0.005778,0.187301
1191,2020-04-29,EOD,5Y,ITRAXX-Europe,33,1,78.02,-0.044619,-1.454349
1190,2020-04-30,EOD,5Y,ITRAXX-Europe,33,1,80.58,0.032285,1.050767
1189,2020-05-01,EOD,5Y,ITRAXX-Europe,33,1,83.58,0.036554,1.189811


In [33]:
# Create a new DataFrame with only 'Date' and 'Standardized_Log_Return'
df_final = df_log_returns[['Date', 'Standardized_Log_Return']].copy()

# Rename the 'Standardized_Log_Return' column to 'ITRAXX-Europe'
df_final.rename(columns={'Standardized_Log_Return': 'ITRAXX-Europe'}, inplace=True)

# Display the resulting DataFrame
df_final.to_csv('data/standardized_itraxx_returns.csv', index=False)